In [1]:
import numpy as np
import pandas as pd

# Importing the Datasets
train = pd.read_csv('readonly/train.csv', encoding="ISO-8859-1", low_memory=False)
test = pd.read_csv('readonly/test.csv', encoding="ISO-8859-1", low_memory=False)

In [2]:
# Columns that are shared between two dataframes
desired_columns = ['inspector_name', 'zip_code', 'disposition', 'fine_amount', 
                   'admin_fee', 'state_fee', 'late_fee']

# Train column
train_columns = desired_columns + ['compliance']

# Columns for onehotencorder
onehot_columns = ['inspector_name', 'zip_code', 'disposition']

# Will be used in the final result as index
result_index = test.loc[:, 'ticket_id']

# Creating view of the existing DataFrame
train = train.loc[:, train_columns]
test = test.loc[:, desired_columns]

In [3]:
### Data cleaning

# Dropping NA or empty values
train.replace('nan', np.nan)
train.dropna(subset=desired_columns + ['compliance'], inplace=True)

# zip_code contained multiple zipcodes that werenot 
# corresponding to our requirements, hence, have been dropped.
train = train[train['zip_code'].str.len() == 5]
train = train[train['zip_code'].str.startswith('48')]

# fine_amount contained multiple missentries of *10 or /10, this is normalizing those errors
train['fine_amount'] = train['fine_amount'].apply(lambda x: x/10 if x>1000 else x)
train['fine_amount'] = train['fine_amount'].apply(lambda x: x*10 if x<10 else x)

train

,inspector_name,zip_code,disposition,fine_amount,admin_fee,state_fee,late_fee,compliance
1,"Williams, Darrin",48208,Responsible by Determination,750.0,20,10,75.0,1.0
6,"Williams, Darrin",48038,Responsible by Default,750.0,20,10,75.0,0.0
7,"Williams, Darrin",48211,Responsible by Default,100.0,20,10,10.0,0.0
8,"Williams, Darrin",48211,Responsible by Default,100.0,20,10,10.0,0.0
9,"Williams, Darrin",48205,Responsible by Default,750.0,20,10,75.0,0.0
...,...,...,...,...,...,...,...,...
250277,"McClain, Melvin",48227,Responsible by Default,500.0,20,10,50.0,0.0
250278,"Zizi, Josue",48235,Responsible by Default,200.0,20,10,20.0,0.0
250287,"Lusk, Gertrina",48227,Responsible by Default,1000.0,20,10,100.0,0.0
250288,"Bell, Maydell",48204,Responsible by Default,500.0,20,10,50.0,0.0


In [4]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer

X = train[desired_columns] # selecting our columns
y_train = train['compliance'] # target column

# Construction of make_column_transformer with specification of columns to be selected for OHE
columns_tran = make_column_transformer((OneHotEncoder(handle_unknown='ignore'), onehot_columns), 
                                       remainder = 'passthrough')
# For clarification purposes seperated fit and transform, could of used fit_transform function
columns_tran.fit(X)
X_train = columns_tran.transform(X) 

X_train.todense()

matrix([[  0.,   0.,   0., ...,  20.,  10.,  75.],
        [  0.,   0.,   0., ...,  20.,  10.,  75.],
        [  0.,   0.,   0., ...,  20.,  10.,  10.],
        ...,
        [  0.,   0.,   0., ...,  20.,  10., 100.],
        [  0.,   0.,   0., ...,  20.,  10.,  50.],
        [  0.,   0.,   0., ...,  20.,  10.,   0.]])

In [5]:
# Models

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV

clr = RandomForestClassifier(n_jobs=-1, random_state=0) # Bad result, confusion matrix result: [131760, 37], [9196, 1316]
# clr = LinearSVC(C=0.1, max_iter=5000) # Worse result

In [6]:
# Data cleaning our test dataset
test.fillna('missing_values', inplace=True)

# Transforming the dataset into OHE matrix
X_test = columns_tran.transform(test)

X_test.todense()

matrix([[  0.,   0.,   0., ...,  20.,  10.,  20.],
        [  0.,   0.,   0., ...,  20.,  10., 100.],
        [  0.,   0.,   0., ...,  20.,  10.,  10.],
        ...,
        [  0.,   0.,   0., ...,  20.,  10.,  10.],
        [  0.,   0.,   0., ...,  20.,  10.,  10.],
        [  0.,   0.,   0., ...,  20.,  10.,   0.]])

In [7]:
## RandomForest
# Previous calculations have been performed and the best depth resulted in being 20 and n_estimators of 100
# depth, n_estimators = [None, 1, 10, 20], [10, 50, 100, 500]
# param = {'n_estimators': [200], 'max_depth': [20]} # RandomForest
# grid_models = GridSearchCV(clr, param_grid=param, scoring='roc_auc')
# grid_models.fit(X_train, y_train)
# Randomforest has been abandond to check for LinearSVC and see the results of it

### LinearSVC
# param = {'C': [0.1, 0.5, 1, 100], 'max_iter': [3000, 5000, 10000]}
# grid_models = GridSearchCV(clr, param_grid=param, scoring='roc_auc')
# grid_models.fit(X_train, y_train)
# # LinearSVC produced worse results: [131776,     21], [ 10253,    259]

In [8]:
# Load previously saved model:
import pickle

with open('gridsearch_result', 'rb') as file:
    grid_models = pickle.load(file)

In [9]:
from sklearn.metrics import confusion_matrix

conf_mat = grid_models.predict(X_train)

confusion = confusion_matrix(y_train, conf_mat)

# The results are not satisfying!
confusion 

array([[131760,     37],
       [  9196,   1316]])

In [10]:
# # Saving the result

# import pickle 
# with open('gridsearch_result', 'wb') as file:
#     pickle.dump(grid_models, file)

In [11]:
# print('Best parameters: ', grid_models.best_params_)

In [12]:
y_scores = grid_models.best_estimator_.predict_proba(X_test)
result = pd.DataFrame(y_scores[:, 1], index = result_index, columns = ['GridSearchCV probablity'])

result

,GridSearchCV probablity
ticket_id,
284932,0.048697
285362,0.036720
285361,0.053711
285338,0.061761
285346,0.085098
...,...
376496,0.022176
376497,0.022176
376499,0.049125
